In [2]:
# ml_env_check.py
# Windows-focused ML environment audit
# Produces a readable report + a JSON file: ml_env_report.json

import json, os, platform, shutil, subprocess, sys, time
from datetime import datetime

def try_import(module_name):
    try:
        return __import__(module_name)
    except Exception:
        return None

def run(cmd):
    try:
        completed = subprocess.run(
            cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
            text=True, shell=True, check=False
        )
        return completed.stdout.strip()
    except Exception as e:
        return f"ERROR: {e}"

def bytes_to_gib(n):
    return round(n / (1024**3), 2)

def gather_os_python():
    return {
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "os": {
            "system": platform.system(),
            "release": platform.release(),
            "version": platform.version(),
            "platform": platform.platform(),
            "machine": platform.machine(),
        },
        "python": {
            "version": sys.version.replace("\n", " "),
            "executable": sys.executable,
        },
        "env_manager": {
            "conda_env": os.environ.get("CONDA_DEFAULT_ENV"),
            "conda_prefix": os.environ.get("CONDA_PREFIX"),
            "venv": os.environ.get("VIRTUAL_ENV"),
        }
    }

def gather_cpu_ram_disk():
    report = {}
    psutil = try_import("psutil")
    if psutil:
        vm = psutil.virtual_memory()
        report["cpu"] = {
            "physical_cores": psutil.cpu_count(logical=False),
            "logical_cores": psutil.cpu_count(logical=True),
            "frequency_mhz": getattr(psutil.cpu_freq(), "current", None)
        }
        report["ram"] = {
            "total_gib": bytes_to_gib(vm.total),
            "available_gib": bytes_to_gib(vm.available),
        }
        # Disk (system drive C:)
        try:
            usage = psutil.disk_usage("C:\\")
            report["disk_C"] = {
                "total_gib": bytes_to_gib(usage.total),
                "used_gib": bytes_to_gib(usage.used),
                "free_gib": bytes_to_gib(usage.free),
                "percent_used": usage.percent
            }
        except Exception as e:
            report["disk_C"] = {"error": str(e)}
    else:
        report["note"] = "Install psutil for detailed CPU/RAM/disk info: pip install psutil"
    # CPU name (Windows often via PROCESSOR_IDENTIFIER)
    report.setdefault("cpu", {})
    report["cpu"]["name"] = os.environ.get("PROCESSOR_IDENTIFIER") or platform.processor()
    return report

def parse_nvidia_smi_text(txt):
    # Pulls driver version and per-GPU model/memory info from nvidia-smi --query
    info = {"driver_version": None, "gpus": []}
    if not txt or "ERROR" in txt:
        return info
    lines = [x.strip() for x in txt.splitlines() if x.strip()]
    # Expect CSV header then rows
    # header: name, driver_version, memory.total, pcie.link.width.max
    if len(lines) >= 2:
        header = [h.strip() for h in lines[0].split(",")]
        for row in lines[1:]:
            cols = [c.strip() for c in row.split(",")]
            item = dict(zip(header, cols))
            if not info["driver_version"]:
                info["driver_version"] = item.get("driver_version")
            # Normalize memory to GiB
            mem = item.get("memory.total")
            mem_gib = None
            if mem:
                # nvidia-smi usually returns "xxxx MiB"
                try:
                    mem_gib = round(float(mem.split()[0]) / 1024.0, 2)
                except Exception:
                    mem_gib = mem
            info["gpus"].append({
                "name": item.get("name"),
                "memory_gib": mem_gib,
                "pcie_width_max": item.get("pcie.link.width.max")
            })
    return info

def gather_cuda_stack():
    # Check presence of NVIDIA stack and CUDA toolkit
    report = {"nvidia_smi_available": shutil.which("nvidia-smi") is not None}
    if report["nvidia_smi_available"]:
        q = "nvidia-smi --query-gpu=name,driver_version,memory.total,pcie.link.width.max --format=csv"
        smi = run(q)
        report.update(parse_nvidia_smi_text(smi))
    else:
        report["note"] = "nvidia-smi not found (no NVIDIA GPU or drivers not installed)"

    # CUDA toolkit (nvcc)
    report["nvcc_available"] = shutil.which("nvcc") is not None
    if report["nvcc_available"]:
        report["nvcc_version_raw"] = run("nvcc --version")

    return report

def gather_pytorch():
    torch = try_import("torch")
    out = {"installed": bool(torch)}
    if not torch:
        return out
    try:
        out["version"] = torch.__version__
        out["cuda_available"] = torch.cuda.is_available()
        out["cuda_device_count"] = torch.cuda.device_count() if out["cuda_available"] else 0
        gpus = []
        if out["cuda_available"]:
            for i in range(torch.cuda.device_count()):
                name = torch.cuda.get_device_name(i)
                cc = torch.cuda.get_device_capability(i)
                mem = torch.cuda.get_device_properties(i).total_memory
                gpus.append({
                    "index": i, "name": name,
                    "compute_capability": f"{cc[0]}.{cc[1]}",
                    "total_mem_gib": bytes_to_gib(mem)
                })
        out["gpus"] = gpus
        # CUDA/cuDNN versions as seen by PyTorch
        out["torch_cuda_version"] = getattr(torch.version, "cuda", None)
        cudnn = getattr(torch.backends, "cudnn", None)
        out["cudnn_available"] = bool(cudnn and cudnn.is_available())
        out["cudnn_version"] = int(cudnn.version()) if out["cudnn_available"] else None
    except Exception as e:
        out["error"] = str(e)
    return out

def gather_tensorflow():
    tf = try_import("tensorflow")
    out = {"installed": bool(tf)}
    if not tf:
        return out
    try:
        out["version"] = tf.__version__
        # GPU logical devices
        try:
            devs = tf.config.list_physical_devices("GPU")
            out["gpu_count"] = len(devs)
            out["gpus"] = [str(d) for d in devs]
        except Exception as e:
            out["gpu_list_error"] = str(e)
        # Build info (CUDA/cuDNN as compiled)
        try:
            bi = tf.sysconfig.get_build_info()
            out["cuda_version_build"] = bi.get("cuda_version")
            out["cudnn_version_build"] = bi.get("cudnn_version")
        except Exception:
            pass
    except Exception as e:
        out["error"] = str(e)
    return out

def gather_key_packages():
    pkgs = ["numpy", "pandas", "scikit-learn", "xgboost", "lightgbm", "torch", "torchvision", "torchaudio", "tensorflow"]
    versions = {}
    for p in pkgs:
        m = try_import(p.replace("-", "_"))
        if m:
            ver = getattr(m, "__version__", None)
            versions[p] = ver or "installed (version unknown)"
        else:
            versions[p] = None
    return versions


report = {}
report["system"] = gather_os_python()
report["resources"] = gather_cpu_ram_disk()
report["nvidia_cuda"] = gather_cuda_stack()
report["pytorch"] = gather_pytorch()
report["tensorflow"] = gather_tensorflow()
report["packages"] = gather_key_packages()

# Print pretty
print("="*70)
print("MACHINE LEARNING ENVIRONMENT REPORT (Windows)".center(70))
print("="*70)
s = report["system"]
print(f"\n[System]")
print(f"  OS: {s['os']['platform']}")
print(f"  Python: {s['python']['version']}")
env = s["env_manager"]
print(f"  Conda env: {env.get('conda_env') or 'None'} | venv: {env.get('venv') or 'None'}")

r = report["resources"]
cpu = r.get("cpu", {})
print(f"\n[CPU/RAM/Disk]")
print(f"  CPU: {cpu.get('name')} | phys/logical: {cpu.get('physical_cores')}/{cpu.get('logical_cores')}")
if "ram" in r:
    print(f"  RAM: total {r['ram']['total_gib']} GiB | available {r['ram']['available_gib']} GiB")
if "disk_C" in r:
    d = r["disk_C"]
    if "error" in d:
        print(f"  Disk C: error: {d['error']}")
    else:
        print(f"  Disk C: total {d['total_gib']} GiB | free {d['free_gib']} GiB ({d['percent_used']}% used)")

print(f"\n[GPU / Drivers / CUDA]")
n = report["nvidia_cuda"]
print(f"  nvidia-smi: {'found' if n.get('nvidia_smi_available') else 'not found'}")
if n.get("driver_version"):
    print(f"  NVIDIA driver: {n['driver_version']}")
if n.get("gpus"):
    for i, g in enumerate(n["gpus"]):
        print(f"  GPU{i}: {g['name']} | {g['memory_gib']} GiB VRAM | PCIe x{g.get('pcie_width_max')}")
print(f"  nvcc (CUDA toolkit): {'found' if n.get('nvcc_available') else 'not found'}")
if n.get("nvcc_version_raw"):
    first = n["nvcc_version_raw"].splitlines()[-1].strip()
    print(f"  nvcc version: {first}")

print(f"\n[PyTorch]")
t = report["pytorch"]
print(f"  Installed: {t['installed']}")
if t["installed"]:
    print(f"  Version: {t.get('version')}")
    print(f"  CUDA available: {t.get('cuda_available')} | devices: {t.get('cuda_device_count')}")
    if t.get("gpus"):
        for g in t["gpus"]:
            print(f"    GPU{g['index']}: {g['name']} | CC {g['compute_capability']} | {g['total_mem_gib']} GiB")
    print(f"  torch CUDA (build): {t.get('torch_cuda_version')}")
    print(f"  cuDNN available: {t.get('cudnn_available')} | version: {t.get('cudnn_version')}")

print(f"\n[TensorFlow]")
tf = report["tensorflow"]
print(f"  Installed: {tf['installed']}")
if tf["installed"]:
    print(f"  Version: {tf.get('version')}")
    print(f"  GPUs seen: {tf.get('gpu_count')}")
    print(f"  Build CUDA: {tf.get('cuda_version_build')} | Build cuDNN: {tf.get('cudnn_version_build')}")

print(f"\n[Key Python Packages]")
for k, v in report["packages"].items():
    print(f"  {k}: {v or 'not installed'}")

# Save JSON report
out_path = os.path.abspath("ml_env_report.json")
with open(out_path, "w", encoding="utf-8") as f:
    json.dump(report, f, indent=2)
print(f"\nReport saved to: {out_path}")
print("\nTip: If PyTorch says CUDA is unavailable but you have an NVIDIA GPU, check matching CUDA Toolkit/driver and install the proper PyTorch build for your CUDA version (or use the CPU build).")



            MACHINE LEARNING ENVIRONMENT REPORT (Windows)             

[System]
  OS: Windows-11-10.0.26100-SP0
  Python: 3.12.11 | packaged by conda-forge | (main, Jun  4 2025, 14:29:09) [MSC v.1943 64 bit (AMD64)]
  Conda env: F:\AI-Ml_conda_env\research | venv: None

[CPU/RAM/Disk]
  CPU: Intel64 Family 6 Model 151 Stepping 2, GenuineIntel | phys/logical: 12/20
  RAM: total 31.76 GiB | available 15.95 GiB
  Disk C: total 399.13 GiB | free 48.05 GiB (88.0% used)

[GPU / Drivers / CUDA]
  nvidia-smi: found
  NVIDIA driver: 576.88
  GPU0: NVIDIA GeForce RTX 5070 Ti | None GiB VRAM | PCIe x16
  nvcc (CUDA toolkit): not found

[PyTorch]
  Installed: True
  Version: 2.7.0+cu128
  CUDA available: True | devices: 1
    GPU0: NVIDIA GeForce RTX 5070 Ti | CC 12.0 | 15.92 GiB
  torch CUDA (build): 12.8
  cuDNN available: True | version: 90701

[TensorFlow]
  Installed: False

[Key Python Packages]
  numpy: 2.3.3
  pandas: 2.3.3
  scikit-learn: not installed
  xgboost: not installed
  lightgbm

In [5]:
# Basic ML Environment Summary (for Jupyter)
# Shows CPU, RAM, GPU, CUDA, PyTorch, TensorFlow info

import platform
import psutil
import torch

def bytes_to_gib(n):
    return round(n / (1024**3), 2)

print("=== BASIC ML ENVIRONMENT SUMMARY ===\n")

# ---- CPU ----
print("[CPU]")
print(f"Name: {platform.processor() or 'Unknown'}")
print(f"Cores: {psutil.cpu_count(logical=False)} physical / {psutil.cpu_count(logical=True)} logical")

# ---- RAM ----
vm = psutil.virtual_memory()
print(f"\n[RAM]")
print(f"Total: {bytes_to_gib(vm.total)} GiB")
# print(f"Available: {bytes_to_gib(vm.available)} GiB")

# ---- GPU / CUDA ----
print("\n[GPU / CUDA]")
if torch.cuda.is_available():
    device_count = torch.cuda.device_count()
    print(f"CUDA available: True")
    for i in range(device_count):
        name = torch.cuda.get_device_name(i)
        props = torch.cuda.get_device_properties(i)
        print(f"GPU {i}: {name}")
        print(f"  Memory: {bytes_to_gib(props.total_memory)} GiB")
        print(f"  Compute Capability: {props.major}.{props.minor}")
else:
    print("CUDA available: False")

# ---- PyTorch ----
print("\n[PyTorch]")
print(f"Version: {torch.__version__}")
print(f"Built with CUDA: {getattr(torch.version, 'cuda', 'None')}")
print(f"cuDNN: {getattr(torch.backends.cudnn, 'version', lambda: None)()}")

# ---- System ----
print("\n[System]")
print(f"OS: {platform.system()} {platform.release()}")
print(f"Python: {platform.python_version()}")

print("\n====================================")


=== BASIC ML ENVIRONMENT SUMMARY ===

[CPU]
Name: Intel64 Family 6 Model 151 Stepping 2, GenuineIntel
Cores: 12 physical / 20 logical

[RAM]
Total: 31.76 GiB

[GPU / CUDA]
CUDA available: True
GPU 0: NVIDIA GeForce RTX 5070 Ti
  Memory: 15.92 GiB
  Compute Capability: 12.0

[PyTorch]
Version: 2.7.0+cu128
Built with CUDA: 12.8
cuDNN: 90701

[System]
OS: Windows 11
Python: 3.12.11

